# Multiple Agents (Group Chat) Demo (Python)

This notebook demonstrates how to interact with **multiple agents** simultaneously using the Python Agent Framework backend API.

## What You'll Learn
- How to send messages to multiple agents at once
- How agents collaborate to answer questions
- Understanding group chat orchestration
- Viewing individual agent contributions
- Working with multi-agent responses

## Prerequisites
Make sure the backend API is running on `http://localhost:8000`

```bash
cd Backend/python
python -m uvicorn main:app --reload
```

## Setup - Install Required Packages

In [ ]:
# Install required packages
!pip install requests python-dotenv -q

import requests
import json
from datetime import datetime
from typing import Dict, Any, Optional, List
import time

print("? Packages installed and modules imported successfully!")

## Configure API Connection

In [ ]:
# API Configuration
API_BASE_URL = "http://localhost:8000"
TIMEOUT = 120  # 2 minutes

# Create session for connection pooling
session = requests.Session()

print(f"? Connected to API: {API_BASE_URL}")

## Step 1: Get Available Agents

In [ ]:
# Get all available agents
response = session.get(f"{API_BASE_URL}/agents")
agents_data = response.json()

print("?? Available Agents for Group Chat:")
print("="*50)

agent_names = []
for agent in agents_data["agents"]:
    name = agent["name"]
    agent_type = agent["type"]
    agent_names.append(name)
    
    print(f"?? {name} ({agent_type})")

print(f"\n? Total agents available: {len(agent_names)}")

## Step 2: Send Message to Multiple Agents (User-Friendly Format)

When you specify multiple agents, the framework orchestrates them to work together on your question.

In [ ]:
# Prepare chat request for multiple agents
multi_agent_request = {
    "message": "What are the latest trends in cloud computing and how do they impact enterprise IT?",
    "agents": ["generic_agent", "knowledge_finder"],  # Multiple agents
    "max_turns": 3,  # Allow up to 3 turns of conversation
    "format": "user_friendly"  # Synthesized response
}

print("?? Sending message to MULTIPLE agents...")
print(f"Message: {multi_agent_request['message']}")
print(f"Agents: {', '.join(multi_agent_request['agents'])}")
print(f"Max Turns: {multi_agent_request['max_turns']}\n")

# Send the request
response = session.post(
    f"{API_BASE_URL}/chat",
    json=multi_agent_request,
    timeout=TIMEOUT
)
response.raise_for_status()
result = response.json()

# Display synthesized response
print("\n?? Synthesized Response:")
print("="*50)
print(f"Primary Agent: {result['agent']}")
print(f"Session ID: {result['session_id']}")
print(f"\nContent:\n{result['content']}")

# Display metadata
if "metadata" in result and result["metadata"]:
    metadata = result["metadata"]
    print(f"\n?? Collaboration Metadata:")
    print(f"   Agent Count: {metadata.get('agent_count', 'N/A')}")
    print(f"   Total Turns: {metadata.get('total_turns', 'N/A')}")
    print(f"   Is Group Chat: {metadata.get('is_group_chat', 'N/A')}")
    
    if "contributing_agents" in metadata:
        contributor_list = ", ".join(metadata["contributing_agents"])
        print(f"   Contributing Agents: {contributor_list}")

## Step 3: Send Message with Detailed Format

The detailed format shows each agent's individual contribution to the conversation.

In [ ]:
# Request with detailed format to see each agent's contribution
detailed_request = {
    "message": "How can AI and machine learning improve customer service in retail?",
    "agents": ["generic_agent", "knowledge_finder"],
    "max_turns": 4,
    "format": "detailed"  # Show individual agent responses
}

print("?? Sending message with DETAILED format...")
print(f"Message: {detailed_request['message']}")
print(f"Agents: {', '.join(detailed_request['agents'])}\n")

response = session.post(
    f"{API_BASE_URL}/chat",
    json=detailed_request,
    timeout=TIMEOUT
)
response.raise_for_status()
detailed_result = response.json()

# Display conversation details
print("\n?? Detailed Group Chat Response:")
print("="*50)
print(f"Conversation ID: {detailed_result['conversation_id']}")
print(f"Total Turns: {detailed_result['total_turns']}")

# Show active participants
participants = detailed_result["active_participants"]
print(f"\n?? Active Participants ({len(participants)}):")
for participant in participants:
    print(f"   - {participant}")

# Show individual agent responses
responses = detailed_result["responses"]
print(f"\n?? Individual Agent Contributions ({len(responses)}):")
print("="*80)

for resp in responses:
    agent_name = resp["agent"]
    content = resp["content"]
    meta = resp["metadata"]
    turn = meta["turn"]
    
    print(f"\n[Turn {turn}] ?? {agent_name}")
    print("-"*80)
    print(content)

# Show summary if available
if "summary" in detailed_result and detailed_result["summary"]:
    print(f"\n?? Conversation Summary:")
    print("="*80)
    print(detailed_result["summary"])

# Metadata
metadata = detailed_result["metadata"]
print(f"\n?? Group Chat Metadata:")
print(f"   Group Chat Type: {metadata.get('group_chat_type')}")
print(f"   Agent Count: {metadata.get('agent_count')}")
print(f"   Response Type: {metadata.get('response_type')}")

## Step 4: Complex Query with Multiple Agents

Try a more complex question that requires different types of expertise.

In [ ]:
# Complex query requiring multiple perspectives
complex_request = {
    "message": """I need help understanding: 
        1. What is Azure OpenAI Service?
        2. How can it be integrated with enterprise applications?
        3. What are the security considerations?""",
    "agents": ["generic_agent", "knowledge_finder"],
    "max_turns": 5,
    "format": "user_friendly"
}

print("?? Sending complex multi-part question...")
print(f"Agents: {', '.join(complex_request['agents'])}\n")

response = session.post(
    f"{API_BASE_URL}/chat",
    json=complex_request,
    timeout=TIMEOUT
)
response.raise_for_status()
complex_result = response.json()

print("\n?? Comprehensive Response:")
print("="*50)
print(complex_result["content"])

# Show collaboration stats
metadata = complex_result["metadata"]
print(f"\n?? Collaboration Statistics:")
print(f"   Agents Participated: {metadata.get('agent_count')}")
print(f"   Total Conversation Turns: {metadata.get('total_turns')}")

if "contributing_agents" in metadata:
    print(f"   Contributors:")
    for contrib in metadata["contributing_agents"]:
        print(f"      - {contrib}")

## Step 5: Compare Single vs Multiple Agent Responses

Let's compare how a single agent vs multiple agents handle the same question.

In [ ]:
test_question = "What are the benefits and challenges of implementing microservices architecture?"

print(f"Test Question: {test_question}")
print("\n" + "="*80)

# Test 1: Single Agent
print("\n?? Test 1: SINGLE AGENT Response")
print("-"*80)

single_agent_req = {
    "message": test_question,
    "agents": ["generic_agent"],
    "format": "user_friendly"
}

single_resp = session.post(
    f"{API_BASE_URL}/chat",
    json=single_agent_req,
    timeout=TIMEOUT
)
single_result = single_resp.json()

print(f"Agent: {single_result['agent']}")
single_content = single_result["content"]
print(f"Response Length: {len(single_content)} characters")
print(f"\n{single_content[:300]}...")

# Test 2: Multiple Agents
print("\n\n?? Test 2: MULTIPLE AGENTS Response")
print("-"*80)

multi_agent_req = {
    "message": test_question,
    "agents": ["generic_agent", "knowledge_finder"],
    "max_turns": 3,
    "format": "user_friendly"
}

multi_resp = session.post(
    f"{API_BASE_URL}/chat",
    json=multi_agent_req,
    timeout=TIMEOUT
)
multi_result = multi_resp.json()

multi_meta = multi_result["metadata"]
print(f"Agents Involved: {multi_meta.get('agent_count')}")
print(f"Total Turns: {multi_meta.get('total_turns')}")

multi_content = multi_result["content"]
print(f"Response Length: {len(multi_content)} characters")
print(f"\n{multi_content[:300]}...")

print("\n\n?? Comparison Summary:")
print("="*80)
print(f"Single Agent: {len(single_content)} chars, 1 agent")
print(f"Multiple Agents: {len(multi_content)} chars, {multi_meta.get('agent_count')} agents, {multi_meta.get('total_turns')} turns")
print(f"\n?? Multiple agents can provide more comprehensive responses through collaboration!")

## Step 6: Get Available Group Chat Templates

The API provides pre-configured templates for common multi-agent scenarios.

In [ ]:
# Get available templates
try:
    response = session.get(f"{API_BASE_URL}/chat/templates")
    templates_data = response.json()
    
    print("?? Available Group Chat Templates:")
    print("="*50)
    
    templates = templates_data["templates"]
    
    for template in templates:
        name = template["name"]
        description = template["description"]
        agent_count = template["agent_count"]
        
        print(f"\n?? Template: {name}")
        print(f"   Description: {description}")
        print(f"   Agents: {agent_count}")
        
        if "agents" in template:
            agent_list = ", ".join(template["agents"])
            print(f"   Agent Names: {agent_list}")
except Exception as ex:
    print(f"??  Templates endpoint not available: {ex}")

## Summary

In this notebook, you learned:
- ? How to send messages to multiple agents simultaneously
- ? Understanding group chat orchestration and collaboration
- ? Viewing individual agent contributions vs synthesized responses
- ? Comparing single-agent vs multi-agent responses
- ? Using detailed format to see the conversation flow
- ? Exploring pre-configured group chat templates

## Key Takeaways
- Multiple agents can provide more comprehensive and nuanced responses
- The framework automatically orchestrates agent collaboration
- You can choose between synthesized (user-friendly) or detailed responses
- Templates provide pre-configured multi-agent scenarios

## Memory Feature (NEW)
You can now enable long-running memory across all agent types:
- Add `enable_memory: True` to any chat request (single or multi-agent)
- Agents will extract and remember your name and persona
- See **04-LongRunningMemory-Demo.ipynb** for complete examples

**Example with Memory:**
```python
{
  "message": "My name is Alice and I'm a data scientist",
  "agents": ["generic_agent", "knowledge_finder"],
  "enable_memory": True,
  "max_turns": 3
}
```

## Next Steps
- Try the **03-ContentSafety-Demo** to see how content is moderated
- Check out the **04-LongRunningMemory-Demo** for memory and session management
- Explore the **single agent demo** for basic agent interaction